In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.fact_sst
USING DELTA
AS
SELECT
  time,
  fk,
  number,
  in_num,
  in_name,
  sst_name,
  peak,
  eval_result,
  result
FROM workspace.gold_staging.stg_sst;

MERGE INTO workspace.gold_serving.fact_sst AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      fk,
      number,
      in_num,
      in_name,
      sst_name,
      peak,
      eval_result,
      result,
      ROW_NUMBER() OVER (
        PARTITION BY time, fk
        ORDER BY time DESC
      ) AS rn
    FROM workspace.gold_staging.stg_sst
  ) sub
  WHERE rn = 1
) AS source
ON target.fk = source.fk
AND target.time = source.time
WHEN MATCHED THEN
  UPDATE SET
    target.in_num = source.in_num,
    target.number = source.number,
    target.in_name = source.in_name,
    target.sst_name = source.sst_name,
    target.peak = source.peak,
    target.eval_result = source.eval_result,
    target.result = source.result
WHEN NOT MATCHED THEN
  INSERT (
    time,
    fk,
    number,
    in_num,
    in_name,
    sst_name,
    peak,
    eval_result,
    result
  )
  VALUES (
    source.time,
    source.fk,
    source.number,
    source.in_num,
    source.in_name,
    source.sst_name,
    source.peak,
    source.eval_result,
    source.result
  );
